In [1]:
# ----------------------------------------------------------------------------
#
# TITLE - potential_fitting.ipynb
# AUTHOR - James Lane
# PROJECT - AST 1501
#
# ----------------------------------------------------------------------------
#
# Docstrings and metadata:
'''Fit the Kuijken & Tremaine model to the MWPotential2014 + triaxial halo model.
'''

__author__ = "James Lane"

In [14]:
### Imports

## Basic
import numpy as np
import sys, os, pdb, importlib

## Plotting
from matplotlib import pyplot as plt

## galpy
from galpy import potential

## Astropy
import astropy.units as apu

## Fitting
from scipy.optimize import curve_fit

## Project specific
sys.path.append('../../src/')
import ast1501.potential

In [3]:
# Matplotlib for notebooks
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# Meta

## Future work

## Information
- Kuijken & Tremaine model: https://ui.adsabs.harvard.edu/#abs/1994ApJ...421..178K/abstract

# Define functions

In [4]:
def kuijken_model(x,vc,alpha,psi0,p):
    
    if alpha != 0:
        axi_component = ( vc**2 / ( 2 * alpha ) ) * ( x[0] / 8 )**( 2 * alpha )
    else:
        axi_component = ( vc**2 ) * np.log( x[0] )
    ##ie
    
    nonaxi_component = ( psi0*( x[0] / 8 )**(p) ) * ( np.cos( 2*( x[1] ) ) )
    
    return axi_component + nonaxi_component
#def

# Make the data to fit
Make MWPotential2014 and replace the normal halo with the triaxial halo

In [5]:
# First declare the data space. Fit a grid from 5-15 kpc in radius and -pi to pi in azimuth
R_limits = [5,15]
phi_limits = [-np.pi,np.pi]
R_range = np.linspace( R_limits[0], R_limits[1], 100 )
phi_range = np.linspace( phi_limits[0], phi_limits[1], 100 )
R_mesh, phi_mesh = np.meshgrid( R_range, phi_range )
mesh_size = R_mesh.shape

# Make into a 2D array that can be fed into the function
Rs = R_mesh.flatten()
phis = phi_mesh.flatten()
xdata = np.vstack((Rs,phis))

In [11]:
importlib.reload(ast1501.potential)

# Make the potential. Some educated guesses about the parameters to 
# make it similar to the fit to MWPotential2014 + triaxial halo. 
# See notebook: kuijken_fitting.ipynb
phi0 = -400
alpha = -0.285
vc = 165
p = -0.5

mypot = ast1501.potential.make_cos2_power_law(phi0, p, alpha, vc)


# pot = np.zeros_like(Rs)
# for i in range( len(pot) ):
#     pot[i] = potential.evaluatePotentials( mwpot_trihalo, R=Rs[i], z=0, phi=phis[i] ).value
# ###i

# pot_inf = potential.evaluatePotentials( mwpot_trihalo, R=100000, z=0, phi=0 ).value
# pot -= pot_inf

TypeError: _call_nodecorator() got multiple values for argument 'phi'

# Example fit

In [ ]:
# Now fit
popt, pcov = curve_fit(kuijken_model, xdata, pot, p0=[220,-1.5,1000,-0.5] )

In [ ]:
variables = ['vc','alpha','psi0','p']
for i in range(len(popt)):
    print( variables[i] + ': ' + str(popt[i]) )
###i

print('\n\n')
print(pcov)

In [ ]:
fig = plt.figure( figsize=(15,4) )

plt.subplot(1, 3, 1)
plt.title("MWPotential2014 + Triaxial Halo")
plt.pcolormesh(R_mesh, phi_mesh, pot.reshape(mesh_size) )
plt.axis( [ R_limits[0], R_limits[1], phi_limits[0], phi_limits[1] ] )
plt.colorbar()

plt.subplot(1, 3, 2)
plt.title("Kuijken Model")
plt.pcolormesh( R_mesh, phi_mesh, kuijken_model(xdata, *popt).reshape(mesh_size) )
plt.axis( [ R_limits[0], R_limits[1], phi_limits[0], phi_limits[1] ] )
plt.colorbar()

plt.subplot(1, 3, 3)
plt.title("Difference")
plt.pcolormesh( R_mesh, phi_mesh, pot.reshape(mesh_size) - kuijken_model(xdata, *popt).reshape(mesh_size) )
plt.axis( [ R_limits[0], R_limits[1], phi_limits[0], phi_limits[1] ] )
plt.colorbar()

pass;

# Fit a range of b

In [ ]:
b_values = np.linspace( 0.7, 1.3, num=14, endpoint=True )
n_bs = len(b_values)
n_variables = 4
variable_storage = np.zeros( ( n_bs, n_variables ) )
variable_names = [r'$v_{c} [km/s]$',r'$\alpha$',r'$\psi_{0}$ [km$^{2}$/s$^{2}$]',r'$p$']

In [ ]:
for i in range( n_bs ):
    
    # Make the galpy data
    mwpot_trihalo = ast1501.potential.make_MWPotential2014_triaxialNFW(halo_b=b_values[i])
    pot = np.zeros_like(Rs)
    for j in range( len(pot) ):
        pot[j] = potential.evaluatePotentials( mwpot_trihalo, R=Rs[j], z=0, phi=phis[j] ).value
    ###i
    pot_inf = potential.evaluatePotentials( mwpot_trihalo, R=100000, z=0, phi=0 ).value
    pot -= pot_inf
    
    # Now fit
    popt, pcov = curve_fit(kuijken_model, xdata, pot, p0=[220,-1.5,1000,-0.5] )
    
    # Store the output
    print(popt)
    variable_storage[i,:] = popt
    
    # Now plot
    fig = plt.figure( figsize=(15,4) )

    plt.subplot(1, 3, 1)
    plt.title("MWPotential2014 + Triaxial Halo")
    plt.pcolormesh(R_mesh, phi_mesh, pot.reshape(mesh_size), cmap='Spectral')
    plt.axis( [ R_limits[0], R_limits[1], phi_limits[0], phi_limits[1] ] )
    plt.xlabel('R [kpc]')
    plt.ylabel(r'$\phi_{GC}$')
    plt.colorbar()

    plt.subplot(1, 3, 2)
    plt.title("Kuijken Model")
    plt.pcolormesh( R_mesh, phi_mesh, kuijken_model(xdata, *popt).reshape(mesh_size), cmap='Spectral' )
    plt.axis( [ R_limits[0], R_limits[1], phi_limits[0], phi_limits[1] ] )
    plt.xlabel('R [kpc]')
    plt.ylabel(r'$\phi_{GC}$')
    plt.colorbar()

    plt.subplot(1, 3, 3)
    plt.title("Difference (% of local amplitude)")
    pot_Ravg_sub = pot.reshape(mesh_size) - np.mean( pot.reshape(mesh_size), 0 ) # Average of each R is subtracted
    pot_Ravg_sub_max = np.amax( pot_Ravg_sub, axis=0 )
    pot_Ravg_sub_max_grid = np.repeat( pot_Ravg_sub_max[:, np.newaxis], pot_Ravg_sub_max.shape[0], axis=1)
    plt.pcolormesh( R_mesh, phi_mesh, 
                   100*np.divide(pot.reshape(mesh_size) - kuijken_model(xdata, *popt).reshape(mesh_size),
                                 pot_Ravg_sub_max_grid), cmap='bwr', vmin=-100, vmax=100 
                  )
    plt.axis( [ R_limits[0], R_limits[1], phi_limits[0], phi_limits[1] ] )
    plt.xlabel('R [kpc]')
    plt.ylabel(r'$\phi_{GC}$')
    plt.colorbar()
    
    plt.suptitle('b = '+str(round(b_values[i],2)) )

    pass;
###i

## Plot the parameter values as a function of b/a

Fit a power law of the form:

$
f(b) = A(b)^{k} + d
$

or 

$
f(b) = A(b-c)^{k} + d
$

In [ ]:
# Now plot the variables

def power_law(x,A,k,d):
    return A*np.power(x,k)+d
#def

def offset_power_law(x,A,c,k,d):
    return A*np.power(x+c,k)+d
#def

def offset_power_law_force(x,A,k,d):
    return A*np.power(x-0.99,k)+d

for i in range( n_variables ):
    
    fig = plt.figure( figsize=(5,5) )
    ax = fig.add_subplot(111)
    
    # Now fit a simple power law
    if i != 3:
        ax.scatter(  b_values, variable_storage[:,i], s=10, color='Black' )
        ax.set_xlabel( 'b/a', fontsize=14 )
        ax.set_ylabel( variable_names[i], fontsize=14 )
        
        popt,pcov = curve_fit( power_law, b_values, variable_storage[:,i], maxfev=10000 )
        fake_x = np.linspace(0.7,1.3,num=100)
        fake_y = power_law(fake_x, *popt)
        ax.plot( fake_x, fake_y, linewidth=0.5, color='Red')
        print('A = '+str(round(popt[0],3)))
        print('k = '+str(round(popt[1],3)))
        print('d = '+str(round(popt[2],3)))
    
    if i == 3:
    
        fit_bounds = ((-np.inf,-0.99,-np.inf,-np.inf),(np.inf,np.inf,np.inf,np.inf))    
        
        ax.scatter(  b_values, variable_storage[:,i], s=10, color='Black' )
        ax.set_xlabel( 'b/a', fontsize=14 )
        ax.set_ylabel( variable_names[i], fontsize=14 )
        
        where_lt_1 = np.where( b_values < 1.0 )[0]
        where_gt_1 = np.where( b_values > 1.0 )[0]
        popt1,pcov1 = curve_fit( offset_power_law, b_values[where_lt_1], 1+variable_storage[where_lt_1,i], 
                                maxfev=10000, bounds=fit_bounds )
        fake_x1 = np.linspace(0.7,1.0,num=100)
        fake_y1 = offset_power_law(fake_x1, *popt1)-1
        ax.plot( fake_x1, fake_y1, linewidth=0.5, color='Red')
        print('b/a less than 1.0')
        print('A = '+str(round(popt1[0],3)))
        print('c = '+str(round(popt1[1],3)))
        print('k = '+str(round(popt1[2],4)))
        print('d = '+str(round(popt1[3],3)))
    
        popt2,pcov2 = curve_fit( offset_power_law_force, b_values[where_gt_1], 1+variable_storage[where_gt_1,i], 
                                maxfev=10000 )
        fake_x2 = np.linspace(1.0,1.3,num=100)
        fake_y2 = offset_power_law_force(fake_x2, *popt2)-1
        ax.plot( fake_x2, fake_y2, linewidth=0.5, color='Red')
        print('b/a greater than 1.0')
        print('A = '+str(round(popt2[0],3)))
        print('c = '+str(round(popt2[0],3)))
        print('k = '+str(round(popt2[1],3)))
        print('d = '+str(round(popt2[2],3)))
    
    plt.show()
    plt.close('all')
###i

pass;

# Fit in radial bins

In [ ]:
R_bin_size = 1.0 # kpc
R_bin_cents = np.arange( R_limits[0], R_limits[1], R_bin_size ) + R_bin_size/2
n_Rs = len(R_bin_cents)

b_values = np.linspace( 0.7, 1.3, num=14, endpoint=True )
n_bs = len(b_values)
n_variables = 4
variable_storage = np.zeros( ( n_bs, n_Rs, n_variables ) )
variable_names = [r'$v_{c} [km/s]$',r'$\alpha$',r'$\psi_{0}$ [km$^{2}$/s$^{2}$]',r'$p$']

In [ ]:
for i in range( n_bs ):
    
    # Make the potential
    mwpot_trihalo = ast1501.potential.make_MWPotential2014_triaxialNFW(halo_b=b_values[i])
    
    for j in range( n_Rs ):
        
        R_hi = R_bin_cents[j]+R_bin_size/2
        R_lo = R_bin_cents[j]-R_bin_size/2
        
        # Make the cutout radial range. Centered on the radial bin
        R_range = np.linspace( R_lo, R_hi, 100 )
        phi_range = np.linspace( phi_limits[0], phi_limits[1], 100 )
        R_mesh, phi_mesh = np.meshgrid( R_range, phi_range )
        mesh_size = R_mesh.shape

        # Make into a 2D array that can be fed into the function
        Rs = R_mesh.flatten()
        phis = phi_mesh.flatten()
        xdata = np.vstack((Rs,phis))
        
        # Make the galpy data
        pot = np.zeros_like(Rs)
        for k in range( len(pot) ):
            pot[k] = potential.evaluatePotentials( mwpot_trihalo, R=Rs[k], z=0, phi=phis[k] ).value
        ###i
        pot_inf = potential.evaluatePotentials( mwpot_trihalo, R=100000, z=0, phi=0 ).value
        pot -= pot_inf

        # Now fit
        popt, pcov = curve_fit(kuijken_model, xdata, pot, p0=[200,-2.8,100,-0.5] )

        # Store the output
        print(popt)
        variable_storage[i,j,:] = popt

        # Now plot
        fig = plt.figure( figsize=(15,4) )

        plt.subplot(1, 3, 1)
        plt.title("MWPotential2014 + Triaxial Halo")
        plt.pcolormesh(R_mesh, phi_mesh, pot.reshape(mesh_size), cmap='Spectral')
        plt.axis( [ R_lo, R_hi, phi_limits[0], phi_limits[1] ] )
        plt.xlabel('R [kpc]')
        plt.ylabel(r'$\phi_{GC}$')
        plt.colorbar()

        plt.subplot(1, 3, 2)
        plt.title("Kuijken Model")
        plt.pcolormesh( R_mesh, phi_mesh, kuijken_model(xdata, *popt).reshape(mesh_size), cmap='Spectral' )
        plt.axis( [ R_lo, R_hi, phi_limits[0], phi_limits[1] ] )
        plt.xlabel('R [kpc]')
        plt.ylabel(r'$\phi_{GC}$')
        plt.colorbar()

        plt.subplot(1, 3, 3)
        plt.title("Difference (% of local amplitude)")
        pot_Ravg_sub = pot.reshape(mesh_size) - np.mean( pot.reshape(mesh_size), 0 ) # Average of each R is subtracted
        pot_Ravg_sub_max = np.amax( pot_Ravg_sub, axis=0 )
        pot_Ravg_sub_max_grid = np.repeat( pot_Ravg_sub_max[:, np.newaxis], pot_Ravg_sub_max.shape[0], axis=1)
        plt.pcolormesh( R_mesh, phi_mesh, 
                       100*np.divide(pot.reshape(mesh_size) - kuijken_model(xdata, *popt).reshape(mesh_size),
                                     pot_Ravg_sub_max_grid), cmap='bwr', vmin=-50, vmax=50 
                      )
        plt.axis( [ R_lo, R_hi, phi_limits[0], phi_limits[1] ] )
        plt.xlabel('R [kpc]')
        plt.ylabel(r'$\phi_{GC}$')
        plt.colorbar()

        plt.suptitle('b = '+str(round(b_values[i],2)) )

        pass;
    ###j
###i

## Make a 2D double power law model

$
f(R,b;A_{1},A_{2},k_{1},k_{2},d) = A_{1}R^{k_{1}} + A_{2}b^{k_{2}} + d
$

In [ ]:
def double_power_law_fit(x,A1,A2,k1,k2,d):
    return A1*(x[0]**k1) + A2*(x[1]**k2) + d
#def

def double_power_law(R,b,A1,A2,k1,k2,d):
    return A1*(R**k1) + A2*(b**k2) + d
#def

In [ ]:
for i in range( n_variables ):
    
    fig = plt.figure( figsize=(17,5) )
    axs = fig.subplots( nrows=1, ncols=3 )
    
    # Now fit a simple power law
    if i < 4:
            
        R_bin_mesh, b_mesh = np.meshgrid(R_bin_cents, b_values)
        fit_mesh_size = R_bin_mesh.shape
        R_bins = R_bin_mesh.flatten()
        bs = b_mesh.flatten()
        fit_data = np.vstack((R_bins,bs))
        
        popt,pcov = curve_fit( double_power_law_fit, fit_data, variable_storage[:,:,i].flatten(), maxfev=10000 )
        fake_z = double_power_law(R_bin_mesh, b_mesh, *popt)
        
        img0 = axs[0].imshow(variable_storage[:,:,i], cmap='viridis', aspect='auto',
                             extent=(np.min(R_bin_cents),np.max(R_bin_cents),np.max(bs),np.min(bs)) )
        axs[0].set_xlabel( 'R [kpc]', fontsize=14 )
        axs[0].set_ylabel( 'b/a', fontsize=14 )
        cbar0 = fig.colorbar(img0, ax=axs[0])
        cbar0.set_label( variable_names[i] )
        
        img1 = axs[1].imshow(fake_z, cmap='viridis', aspect='auto',
                             extent=(np.min(R_bin_cents),np.max(R_bin_cents),np.max(bs),np.min(bs)) )
        axs[1].set_xlabel( 'R [kpc]', fontsize=14 )
        axs[1].set_ylabel( 'b/a', fontsize=14 )
        cbar1 = fig.colorbar(img1, ax=axs[1])
        cbar1.set_label( variable_names[i] )
        
        img2 = axs[2].imshow( 100*np.divide( fake_z-variable_storage[:,:,i], variable_storage[:,:,i] ) , 
                             cmap='Spectral', aspect='auto', 
                             extent=(np.min(R_bin_cents),np.max(R_bin_cents),np.max(bs),np.min(bs)) )
        axs[2].set_xlabel( 'R [kpc]', fontsize=14 )
        axs[2].set_ylabel( 'b/a', fontsize=14 )
        cbar2 = fig.colorbar(img2, ax=axs[2])
        cbar2.set_label( 'Percent difference' )
        
        fig.subplots_adjust(wspace=0.5)
        
        # Print the results
        print('1 are for R')
        print('2 are for b')
        print('A1 = '+str(round(popt[0],4)))
        print('A2 = '+str(round(popt[1],4)))
        print('k1 = '+str(round(popt[2],4)))
        print('k2 = '+str(round(popt[3],4)))
        print('d = '+str(round(popt[4],4)))
        print(popt)
        
    ##fi
    
    plt.show()
    plt.close('all')
###i

## Make a 2D quadratic model

$
f(R,b;A_{1},A_{2},A_{3},A_{4},d) = A_{1}R + A_{2}R^{2} + A_{3}b + A_{4}b^{2} + d
$

In [ ]:
def double_quadratic_fit(x,A1,A2,A3,A4,d):
    return A1*x[0] + A2*x[0]**2 + A3*x[1] + A4*x[1]**2 + d
#def

def double_quadratic(R,b,A1,A2,A3,A4,d):
    return A1*R + A2*R**2 + A3*b + A4*b**2 + d
#def

In [ ]:
for i in range( n_variables ):
    
    fig = plt.figure( figsize=(17,5) )
    axs = fig.subplots( nrows=1, ncols=3 )
    
    # Now fit a simple power law
    if i < 4:
            
        R_bin_mesh, b_mesh = np.meshgrid(R_bin_cents, b_values)
        fit_mesh_size = R_bin_mesh.shape
        R_bins = R_bin_mesh.flatten()
        bs = b_mesh.flatten()
        fit_data = np.vstack((R_bins,bs))
        
        popt,pcov = curve_fit( double_quadratic_fit, fit_data, variable_storage[:,:,i].flatten(), maxfev=10000 )
        fake_z = double_quadratic(R_bin_mesh, b_mesh, *popt)
        
        img0 = axs[0].imshow(variable_storage[:,:,i], cmap='viridis', aspect='auto',
                             extent=(np.min(R_bin_cents),np.max(R_bin_cents),np.max(bs),np.min(bs)) )
        axs[0].set_xlabel( 'R [kpc]', fontsize=14 )
        axs[0].set_ylabel( 'b/a', fontsize=14 )
        cbar0 = fig.colorbar(img0, ax=axs[0])
        cbar0.set_label( variable_names[i] )
        
        img1 = axs[1].imshow(fake_z, cmap='viridis', aspect='auto',
                             extent=(np.min(R_bin_cents),np.max(R_bin_cents),np.max(bs),np.min(bs)) )
        axs[1].set_xlabel( 'R [kpc]', fontsize=14 )
        axs[1].set_ylabel( 'b/a', fontsize=14 )
        cbar1 = fig.colorbar(img1, ax=axs[1])
        cbar1.set_label( variable_names[i] )
        
        img2 = axs[2].imshow( 100*np.divide( fake_z-variable_storage[:,:,i], variable_storage[:,:,i] ) , 
                             cmap='Spectral', aspect='auto', 
                             extent=(np.min(R_bin_cents),np.max(R_bin_cents),np.max(bs),np.min(bs)) )
        axs[2].set_xlabel( 'R [kpc]', fontsize=14 )
        axs[2].set_ylabel( 'b/a', fontsize=14 )
        cbar2 = fig.colorbar(img2, ax=axs[2])
        cbar2.set_label( 'Percent difference' )
        
        fig.subplots_adjust(wspace=0.5)
        
        # Print the results
        print('A1 = '+str(round(popt[0],4)))
        print('A2 = '+str(round(popt[1],4)))
        print('A3 = '+str(round(popt[2],4)))
        print('A4 = '+str(round(popt[3],4)))
        print('d = '+str(round(popt[4],4)))
        print(popt)
        
    ##fi
    
    plt.show()
    plt.close('all')
###i